# 10 - Systems of Equations
- 10.1 Algebra and geometry of equations
- 10.2 From systems to matrices
- 10.3 Row reduction
- 10.4 Gaussian elimination
- 10.5 Row-reduced echelon form
- 10.6 Gauss-Jordan elimination
- 10.7 Possibilities for solutions
- 10.8 Matrix spaces, row reduction
- 10.9 Exercises
- 10.10 Answers
- 10.11 Coding challenges
- 10.12 Code solutions

In [1]:
import numpy as np
import sympy as sym

## 10.1 Algebra and geometry of equations
Every solution to the system of equations $Ax = b$ has one of the following outcomes:

- If $A$ is nonsingular, then there is exactly one (unique) solution
    - $b \in \text{span}(A)$ and linearly independent
    - Geometric interpretation: Lines intersect at the point $x$ with coordinates equal to the solution
- If $A$ is singular and $b \in \text{span}(A)$, then there are an infinite number of solutions
    - Geometric interpretation: Colinear lines
- If $A$ is singular and $b \notin \text{span}(A)$, then there is no solution
    - Geometric interpretation: Parallel lines

Note
- Existence of unique solution depends only on $A$


### Singularity
A nonsingular matrix $A$ has the following properties:
1. There exists $A^{-1}$ such that $A^{-1} A = A A^{-1} = I$
2. $\text{det}(A) \neq 0$
3. $\text{rank}(A) = n$ where $A$ is $n \times n$ matrix
4. Null space of matrix is empty $N(A) = \emptyset$ e.g. $Az \neq 0$ for any vector $z \neq 0$

## 10.2 From systems to matrices
Terminology used in system of equations $Ax = b$

- Variables: $x$
    - Unknowns that solve the system
- Coefficients: $A$
    - Matrix of coefficients, one equation per row
- Constants: $b$
    - Right hand side, do not participate in multiplication $Ax$

## 10.3 Row reduction

### Row Echelon Form aka REF
Row echelon form of a matrix $\text{REF}(A)$ corresponds to an upper triangular matrix

Example
$$
\text{REF}(A) = 
\begin{bmatrix}
a & b & c \\
0 & d & e \\
0 & 0 & f \\
\end{bmatrix}
$$

Notes
- Leftmost nonzero elements in each row are known as pivots
- REF form of a matrix is not unique

### Elementary Row Operations (ERO)
Use elementary row operations (listed below) to transform a matrix to row echelon form.

1. Swap rows
2. Scale a row by a nonzero constant
3. Add to a row the scalar multiple of another row

Notes
- Matrix rank is equal to...
    - ... number of nonzero rows in row echelon form
    - ... number of pivots
- ERO do not change the row space of the matrix, hence do not change the matrix rank

### Why Row Echelon Form (REF)?
Row echelon form allows us to solve a system of equations $Ux = b$ for $x$ using back substitution.
- Solves for $x$ moving from bottom to top.

$$
x_i = \frac{b_i - \sum_{j=i+1}^{n} U_{i,j} x_j}{U_{i,i}} \quad i = n, n-1, \dots, 1
$$

Forward substitution solves a system of equations $Lx = b$ for $x$.
- Solves for $x$ moving from top to bottom.

$$
x_i = \frac{b_i - \sum_{j=1}^{i-1} L_{i,j} x_j}{L_{i,i}} \quad i = 1, 2, \dots, n
$$

## 10.4 Gaussian elimination

### LU Decomposition
LU decomposition solves the linear system of equations $Ax = b$ by factorizing matrix $A$ into lower triangular matrix $L$ and upper triangular matrix $U$.
- Factorization refers to rewriting a mathematical object as the product of other objects

1. Factorize $A$ to product $LU$
2. Rewrite $Ax = b$ as $LUx = b$
3. Solve $Ly = b$ for $y$
4. Solve $Ux = y$ for $x$

Notes
- Technique suitable only for square matrices eg $A$ is a $n \times n$ matrix
- Since factorization is independent of the right hand side, it can be performed once and reused for multiple right hand sides

### Gaussian Elmination
Gaussian Elimination solves the linear system of equations $Ax = b$ using REF.

1. Augment $A$ with $b$ to produce the matrix $[ A | b ]$
2. Use elementary row operations to transform augmented matrix to row echelon form $\text{REF}([ A | b ]) = U$
3. Use back substitution to solve upper triangular system $Ux$ for $x$

## 10.5 Row-reduced echelon form

### Row-reduced Echelon Form aka RREF
Row-reduced echelon form takes echelon form a little further by 
1. Rescaling nonzero rows by a scalar constant so that pivots have a value of 1
2. Applying row reduction to the remaining rows so that the pivot element is the only nonzero element in each column

Example
$$
\text{RREF}(A) = 
\begin{bmatrix}
1 & 0 & a \\
0 & 1 & b \\
0 & 0 & 0 \\
\end{bmatrix}
$$

In [2]:
A = np.array([[1,2,3],[4,5,6],[7,8,9]])

print(A)

# Verify that the number of pivots (2) is equal to the matrix rank.
rrefA, _ = sym.Matrix(A).rref()
print(np.array(rrefA))
np.testing.assert_equal(np.linalg.matrix_rank(A), 2)

[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[1 0 -1]
 [0 1 2]
 [0 0 0]]


## 10.6 Gauss-Jordan elimination

### Gauss-Jordan Elimination
Gauss-Jordan Elimination solves the linear system of equations $Ax = b$ using RREF.

1. Augment $A$ with $b$ to produce the matrix $[ A | b ]$
2. Use elementary row operations to transform augmented matrix $\text{RREF}([ A | b ]) = [ I | x ]$

Since Gauss-Jordan uses RREF, using back substitution to solve for $x$ is not required. 


## 10.7 Possibilities for solutions
(_Repeat from 10.1_)

Every solution to the system of equations $Ax = b$ has one of the following outcomes:
1. System has no solutions
    - $A$ is singular
    - $A$ is rank deficient
    - $b$ does not span $A$
2. System has exactly one solution
    - $A$ is nonsingular
    - $A$ is full rank
    - $\text{RREF}(A) = I$
    - $b$ spans $A$ and is linearly independent
    - $N(A) = \emptyset$
3. System has an infinite number of solutions
    - $A$ is singular
    - $A$ is rank deficient
    - $b$ spans $A$ and is linearly dependent

## 10.8 Matrix spaces, row reduction
- ERO does not change rank of the matrix
- ERO does not change the row space of the matrix
- ERO might change the column space of the matrix

## 10.11 Coding challenges

> 2. The margin notes in section 10.5 describe patterns in the RREF of a matrix that depend on the size and shape of the matrix. The idea is that every RREF is essentially the identity matrix, possibly with additional rows of zeros or columns of nonzero numbers. Create random matrices that are (1) square, (2) wide, and (3) tall and compute and comment on the RREF of those matrices.

In [3]:
# Square matrix
# RREF(A) = I
# Matrix is 
m, n = 4, 4
A = np.random.random((m, n))
rrefA, _ = sym.Matrix(A).rref()
print("square:\n", np.array(rrefA))

# Wide matrix
# RREF(A) = | I | X |
m, n = 4, 6
A = np.random.random((m, n))
rrefA, _ = sym.Matrix(A).rref()
print("wide:\n", np.array(rrefA))

# Tall matrix
# RREF(A) = | I |
#           | 0 |
m, n = 6, 4
A = np.random.random((m, n))
rrefA, _ = sym.Matrix(A).rref()
print("tall:\n", np.array(rrefA))

square:
 [[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]
wide:
 [[1 0 0 0 3.04783851597072 -0.737573763658058]
 [0 1 0 0 -6.28477867787247 2.32641474739729]
 [0 0 1 0 0.577227709280858 0.543645078674051]
 [0 0 0 1 0.504076602318774 0.0402422527658611]]
tall:
 [[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 0 0]
 [0 0 0 0]]
